# Analysis of GDELT violence data

## Query

```sql
SELECT Year, 
	   ActionGeo_CountryCode AS CountryCode,
	   EventRootCode,
	   EventCode,
	   COUNT(GLOBALEVENTID) AS SumEvents,
	   ANY_VALUE(GoldsteinScale) AS GoldsteinScale,
	   AVG(NumMentions) as AvgNumMentions,
	   SUM(NumMentions) as SumNumMentions,
	   AVG(AvgTone) as AvgAvgTone,
FROM `gdelt-bq.full.events`
WHERE 
	EventRootCode IN ("17", "18", "19") 
	AND Year >= 1979
GROUP BY Year, ActionGeo_CountryCode, EventRootCode, EventCode
ORDER BY Year
```

In [1]:
import pandas as pd
import numpy as np

### Get CAMEO event codes. 

In [2]:
!curl -o data/CAMEO_codes.txt https://www.gdeltproject.org/data/lookups/CAMEO.eventcodes.txt 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12157  100 12157    0     0  11768      0  0:00:01  0:00:01 --:--:-- 11780


In [3]:
cameo_codes = pd.read_csv('data/CAMEO_codes.txt', sep='\t', 
                          names=['code', 'descr'], skiprows=1,
                         dtype=str)

cameo_codes['code'] = cameo_codes['code'].astype(str)

cameo_codes.head()

,code,descr
0,01,MAKE PUBLIC STATEMENT
1,010,"Make statement, not specified below"
2,011,Decline comment
3,012,Make pessimistic comment
4,013,Make optimistic comment


### Load query data. 

In [4]:
data = pd.read_csv('data/query_result.csv')
data['EventCode'] = data['EventCode'].astype(str)
data['EventRootCode'] = data['EventRootCode'].astype(str)

if 'f0_' in data.columns:
    data.rename({'f0_': 'GoldsteinScale'},
                axis=1, inplace=True)

data.shape

(179946, 9)

In [5]:
data.head()

,Year,CountryCode,EventRootCode,EventCode,SumEvents,GoldsteinScale,AvgNumMentions,SumNumMentions,AvgAvgTone
0,1979,CH,19,193,350,-10.0,3.988571,1396,1.906000
1,1979,HR,19,190,14,-10.0,2.642857,37,4.335594
2,1979,SU,17,173,16,-5.0,6.000000,96,3.358047
3,1979,SY,19,193,71,-10.0,3.647887,259,3.557653
4,1979,GM,17,172,33,-5.0,4.484848,148,4.841399


### Merge event and event root description. 

In [6]:
data = data.merge(cameo_codes, left_on='EventCode', right_on='code', how='left')
data = data.merge(cameo_codes, left_on='EventRootCode', right_on='code', how='left')

data.drop(['code_x', 'code_y'], axis=1, inplace=True)

data.rename({'descr_x': 'EventDescr',
             'descr_y': 'EventRootDescr'},
            axis=1, inplace=True)

In [7]:
data.head()

,Year,CountryCode,EventRootCode,EventCode,SumEvents,GoldsteinScale,AvgNumMentions,SumNumMentions,AvgAvgTone,EventDescr,EventRootDescr
0,1979,CH,19,193,350,-10.0,3.988571,1396,1.906000,Fight with small arms and light weapons,FIGHT
1,1979,HR,19,190,14,-10.0,2.642857,37,4.335594,"Use conventional military force, not specified...",FIGHT
2,1979,SU,17,173,16,-5.0,6.000000,96,3.358047,"Arrest, detain, or charge with legal action",COERCE
3,1979,SY,19,193,71,-10.0,3.647887,259,3.557653,Fight with small arms and light weapons,FIGHT
4,1979,GM,17,172,33,-5.0,4.484848,148,4.841399,"Impose administrative sanctions, not specified...",COERCE


### Some countries are NULL. 

In [8]:
data.isnull().sum()

Year                 0
CountryCode       1386
EventRootCode        0
EventCode            0
SumEvents            0
GoldsteinScale       0
AvgNumMentions       0
SumNumMentions       0
AvgAvgTone           0
EventDescr           0
EventRootDescr       0
dtype: int64

### Download and merge country names. 

In [9]:
!curl -o data/country_names.csv https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5064  100  5064    0     0  19627      0 --:--:-- --:--:-- --:--:-- 19627


In [10]:
cn = pd.read_csv('data/country_names.csv')
cn = cn.drop('ISO 3166', axis=1)
cn.columns = ['CountryCode', 'CountryName']
cn.head()

,CountryCode,CountryName
0,AF,Afghanistan
1,AX,Akrotiri
2,AL,Albania
3,AG,Algeria
4,AQ,American Samoa


In [11]:
data = data.merge(cn, on='CountryCode', how='left')

### Downloading and merging total values. 

In [12]:
!curl -o data/yearly_country_totals.csv http://data.gdeltproject.org/normfiles/yearly_country.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  136k  100  136k    0     0   145k      0 --:--:-- --:--:-- --:--:--  145k


In [13]:
totals = pd.read_csv('data/yearly_country_totals.csv', names=['Year', 'CountryCode', 'TotalEvents'])
totals.head()

,Year,CountryCode,TotalEvents
0,1920,NaN,13636
1,1920,AA,3
2,1920,AC,63
3,1920,AE,1585
4,1920,AF,3523


In [14]:
data = data.merge(totals, on=['Year', 'CountryCode'], how='left')

### Compute normalized events count (over 1000 events). 

In [15]:
data['NormalizedEvents1000'] = data['SumEvents'] / data['TotalEvents'] * 10**3

### Removing rows with NULL values. 

In [16]:
before = data.shape[0]
data = data.dropna()
after = data.shape[0]
removed = before - after
percent_removed = round(removed / before, 2) * 100
print(f"Removed {before - after} rows ({percent_removed}% of {before} rows.)")

Removed 4910 rows (3.0% of 181342 rows.)


In [17]:
data.isnull().sum().sum()

0

In [18]:
data.shape

(176432, 14)

### Reordering columns.

In [19]:
prev = len(data.columns)
data = data[['Year', 'CountryCode', 'CountryName', 
             'SumEvents', 'TotalEvents', 'NormalizedEvents1000', 
             'EventRootCode', 'EventRootDescr',
             'EventCode', 'EventDescr', 
             'GoldsteinScale', 'AvgNumMentions', 'SumNumMentions', 'AvgAvgTone'
             ]]

assert len(data.columns) == prev

In [20]:
data.head()

,Year,CountryCode,CountryName,SumEvents,TotalEvents,NormalizedEvents1000,EventRootCode,EventRootDescr,EventCode,EventDescr,GoldsteinScale,AvgNumMentions,SumNumMentions,AvgAvgTone
0,1979,CH,China,350,33541,10.434990,19,FIGHT,193,Fight with small arms and light weapons,-10.0,3.988571,1396,1.906000
1,1979,HR,Croatia,14,714,19.607843,19,FIGHT,190,"Use conventional military force, not specified...",-10.0,2.642857,37,4.335594
2,1979,SU,Sudan,16,1354,11.816839,17,COERCE,173,"Arrest, detain, or charge with legal action",-5.0,6.000000,96,3.358047
3,1979,SY,Syria,71,3119,22.763706,19,FIGHT,193,Fight with small arms and light weapons,-10.0,3.647887,259,3.557653
4,1979,GM,Germany,33,6615,4.988662,17,COERCE,172,"Impose administrative sanctions, not specified...",-5.0,4.484848,148,4.841399


In [21]:
data.isna().sum().sum()

0

## Export

In [22]:
data.to_csv('gdelt_conflict/gdelt_conflict_1_0.csv', index=None)